<a href="https://colab.research.google.com/github/iam-abbas/Awesome-Profile-README-templates/blob/master/rStocks_Trends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance praw==7.0

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import re
import praw
import time
from collections import Counter
from functools import reduce
from operator import add
from typing import Set
import json

In [ ]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'must', 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [ ]:
block_words = ["DIP", "", "$", "RH", "YOLO", "PORN", "BEST", "MOON", "HOLD", "FAKE", "HEDGE", "FUND", "WISH", "USD", "BUY", "SELL", "EV", "MARK", "RELAX", "LOL", "LMAO", "LMFAO", "EPS", "DCF", "NYSE", "FTSE", "APE", "CEO", "CTO", "FUD", "DD", "AM", "PM", "FDD", "EDIT", "TA", "UK", "AMC", "GME"]

In [ ]:
with open('tickers.json') as tickerFile:
    tickerList = json.load(tickerFile)

In [ ]:
def verify_ticker(tic):
    try:
        if tickerList[tic]:
            return True
    except:
        pass
    return False

In [ ]:
def extract_ticker(body: str, re_string: str = "[$][A-Za-z]*|[A-Z][A-Z]{1,}") -> Set[str]:
    """Simple Regex to get tickers from text."""
    ticks = set(re.findall(re_string, str(body)))
    res = set()
    for item in ticks:
        if item not in block_words and item.lower() not in stop_words and item:
            try:
                tic = item.replace("$", "").upper()
                if verify_ticker(tic):
                  res.add(tic)
            except Exception as e:
                print(e)
    return res

In [ ]:
reddit = praw.Reddit(
    client_id="9Aq-wTeGLJBKsQ", 
    client_secret="EclWNx5qOyIZLiRkd10Oln0iNPUXvQ", 
    user_agent="ScrapeStocks"
)

Version 7.0.0 of praw is outdated. Version 7.1.4 was released 1 day ago.


In [ ]:
current_time = int(time.time())

In [ ]:
def userReliability(author):
  linkKarma = author.link_karma
  commentKarma = author.comment_karma
  isVerified = 1 if author.has_verified_email else 0.2
  suspicious = 1
  currentTickers = []
  for comment in author.comments.new(limit=10):
    tickers = list(extract_ticker(comment.body))
    for ticker in tickers:
      if verify_ticker(ticker):
        if ticker in currentTickers or ticker.lower() in comment.body.lower():
          suspicious -= 0.1
        else:
          suspicious += 0.1
          currentTickers.append(ticker)
  karmaPower = ((linkKarma*1.1)+(commentKarma*1.25))/2000
  userMeasure = karmaPower*isVerified*suspicious
  # print("Suspicious:", suspicious, "karmaPower:", karmaPower, 'isVerified:', isVerified)
  return userMeasure




In [ ]:
_posts = []

new_bets = reddit.subreddit("pennystocks+robinhoodpennystocks").new(limit=None)
new_bets

for post in new_bets:
    post_age = (current_time - post.created_utc) / 60 / 60 / 48
    flair = post.link_flair_text if post.link_flair_text else "None"
    if post_age < 1 and any(sub in flair for sub in ["DD", "News", "Discussion", "Catalyst", "Tips", "Stock"]):
        reliability = userReliability(post.author)
        if reliability > 0.75:
          _posts.append(
              [
                  post.id,
                  post.title,
                  post.score,
                  post.num_comments,
                  post.upvote_ratio,
                  post.total_awards_received,
                  post.link_flair_text,
                  reliability
              ]
          )

# create a dataframe
_posts = pd.DataFrame(
    _posts,
    columns=[
        "id",
        "title",
        "score",
        "comments",
        "upvote_ratio",
        "total_awards",
        "flair",
        "author_reliability"
    ],
)

In [ ]:
_posts

,id,title,score,comments,upvote_ratio,total_awards,flair,author_reliability
0,lg0b72,"$AKBA Bio-pharma stock DD, upcoming catalyst",1,1,1.00,0,DD :DD:,0.941098
1,lg07u2,Improved: Algorithm that shows currently trend...,3,2,0.80,0,Discussion,3.340125
2,lfzgro,GRPS is now Pink!!,6,6,0.80,1,Catalyst,1.190520
3,lfytwy,"[DD] On Abeona Therapeutics $ABEO, with upcomi...",26,12,0.94,0,DD :DD:,1.354990
4,lfyikp,WALRF good swing play. (EV play),5,4,0.73,0,General Discussion,4.731350
...,...,...,...,...,...,...,...,...
121,lenmhf,(GTII) The Great OTC SHORT SQUEEZE of 2021,49,70,0.89,2,DD :DD:,0.908363
122,len9kc,HIPH - Catalyst w DD,11,10,0.76,0,Catalyst,11.450975
123,lemwn2,ALYI and Hemp batteries.,15,17,0.83,0,General Discussion,2.661775
124,lemkx4,Undiscovered Solar Play,9,15,0.63,0,Stock Info :stonk:,3.244850


In [ ]:
_posts["Tickers"] = _posts.title.apply(extract_ticker)

In [ ]:
_posts

,id,title,score,comments,upvote_ratio,total_awards,flair,author_reliability,Tickers
0,lg0b72,"$AKBA Bio-pharma stock DD, upcoming catalyst",1,1,1.00,0,DD :DD:,0.941098,{AKBA}
1,lg07u2,Improved: Algorithm that shows currently trend...,3,2,0.80,0,Discussion,3.340125,{}
2,lfzgro,GRPS is now Pink!!,6,6,0.80,1,Catalyst,1.190520,{GRPS}
3,lfytwy,"[DD] On Abeona Therapeutics $ABEO, with upcomi...",26,12,0.94,0,DD :DD:,1.354990,{ABEO}
4,lfyikp,WALRF good swing play. (EV play),5,4,0.73,0,General Discussion,4.731350,{WALRF}
...,...,...,...,...,...,...,...,...,...
121,lenmhf,(GTII) The Great OTC SHORT SQUEEZE of 2021,49,70,0.89,2,DD :DD:,0.908363,{GTII}
122,len9kc,HIPH - Catalyst w DD,11,10,0.76,0,Catalyst,11.450975,{HIPH}
123,lemwn2,ALYI and Hemp batteries.,15,17,0.83,0,General Discussion,2.661775,{ALYI}
124,lemkx4,Undiscovered Solar Play,9,15,0.63,0,Stock Info :stonk:,3.244850,{}


In [ ]:
_posts.title.drop_duplicates(keep='first', inplace=True)

In [ ]:
ticker_sets = _posts.Tickers.to_list()
counts = {}
for s in ticker_sets:
  for tic in s:
    if tic in counts:
      counts[tic] += 1
    else:
      counts[tic] = 1

In [ ]:
counts

{'ABEO': 1,
 'ACRX': 1,
 'AG': 1,
 'AGTC': 2,
 'AITX': 1,
 'AKBA': 1,
 'ALYI': 1,
 'ARTM': 1,
 'ASCK': 1,
 'ASRT': 1,
 'ATHE': 1,
 'BBRRF': 1,
 'BBRW': 1,
 'BIG': 1,
 'BIOL': 2,
 'BRPA': 1,
 'BSQR': 1,
 'CBBT': 1,
 'CHEK': 1,
 'CLSN': 1,
 'CN': 2,
 'CVS': 1,
 'DKMR': 1,
 'DPLS': 1,
 'EAST': 3,
 'ERPNF': 1,
 'ETFM': 1,
 'GHSI': 1,
 'GNUS': 1,
 'GRPS': 1,
 'GTE': 1,
 'GTEH': 1,
 'GTII': 1,
 'HIPH': 1,
 'INND': 2,
 'IQST': 1,
 'ITP': 1,
 'ITRM': 1,
 'KALTF': 1,
 'KAVL': 1,
 'KIQ': 1,
 'LVVV': 1,
 'MBRX': 1,
 'MINE': 1,
 'MITI': 1,
 'MITT': 1,
 'MJ': 1,
 'MJLB': 1,
 'NAK': 1,
 'NAKD': 2,
 'NBY': 1,
 'NECA': 1,
 'NEW': 1,
 'NEXT': 1,
 'NLST': 1,
 'NOVC': 1,
 'NXGWF': 1,
 'OCGN': 2,
 'OTLK': 1,
 'OZSC': 3,
 'PBTS': 1,
 'PKG': 2,
 'PLAY': 1,
 'PLX': 1,
 'PTNYF': 2,
 'REI': 2,
 'RLFTF': 1,
 'SENY': 1,
 'SFIO': 1,
 'SGMD': 1,
 'SONM': 1,
 'SRGA': 2,
 'SSFT': 1,
 'TBLT': 1,
 'TLOFF': 1,
 'TNXP': 2,
 'TRXC': 1,
 'TYME': 1,
 'UBMRF': 1,
 'VISL': 1,
 'WALRF': 1,
 'XOM': 2,
 'ZOM': 1,
 'ZSAN': 3}

In [ ]:
tick_df = pd.DataFrame(counts.items(), columns=["Ticker", "Mentions"])

In [ ]:
tick_df

,Ticker,Mentions
0,AKBA,1
1,GRPS,1
2,ABEO,1
3,WALRF,1
4,NAKD,2
...,...,...
79,MINE,1
80,GTII,1
81,HIPH,1
82,ALYI,1


In [ ]:
tick_df.sort_values(by=["Mentions"], inplace=True, ascending=False)
tick_df.reset_index(inplace=True, drop=True)
tick_df

,Ticker,Mentions
0,OZSC,3
1,ZSAN,3
2,EAST,3
3,TNXP,2
4,INND,2
...,...,...
79,NECA,1
80,NLST,1
81,SONM,1
82,SENY,1


In [ ]:
tick_df = tick_df.head(25)

In [ ]:
def calculate_change(start: float, end: float) -> float:
    """Use Yahoo Finance API to get the relevent data."""
    return round(((end - start) / start) * 100, 2)


def get_change(ticker: str, period: str = "1d") -> float:
    return calculate_change(yf.Ticker(ticker).history(period)["Open"].to_list()[0],
                            yf.Ticker(ticker).history(period)["Close"].to_list()[-1])
    
dataColumns = ["Name", "Industry", "Previous Close", "5d Low", "5d High", "1d Change (%)", "5d Change (%)", "1mo Change (%)"]

def getTickerInfo(ticker):
    
  # Standard Data
  info = yf.Ticker(ticker).info
  tickerName = info["longName"]
  try:
    tickerIndustry = info["industry"]
  except:
    tickerIndustry = "N/A"

  # previous Day close
  tickerClose = yf.Ticker(ticker).history(period="1d")["Close"].to_list()[-1]

  # Highs and Lows
  highLow = yf.Ticker(ticker).history(period="5d")
  Low5d = min(highLow["Low"].to_list())
  High5d = max(highLow["High"].to_list())

  # Changes
  change1d = get_change(ticker)
  change5d = get_change(ticker, "5d")
  change1mo = get_change(ticker, "1mo")

  return pd.Series([tickerName, tickerIndustry, tickerClose, Low5d, High5d, change1d, change5d, change1mo])


In [ ]:
tick_df[dataColumns] = tick_df.Ticker.apply(getTickerInfo)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
tick_df

,Ticker,Mentions,Name,Industry,Previous Close,5d Low,5d High,1d Change (%),5d Change (%),1mo Change (%)
0,OZSC,3,"Ozop Energy Solutions, Inc.",Electrical Equipment & Parts,0.378400,0.148000,0.385000,51.42,122.07,2456.76
1,ZSAN,3,Zosano Pharma Corporation,Biotechnology,1.930000,0.940000,1.950000,72.32,93.97,129.76
2,EAST,3,"Eastside Distilling, Inc.",Beverages—Wineries & Distilleries,1.770000,1.430000,1.940000,-0.62,18.79,27.34
3,TNXP,2,Tonix Pharmaceuticals Holding Corp.,Biotechnology,1.550000,0.950000,1.660000,-6.63,56.57,68.48
4,INND,2,"InnerScope Hearing Technologies, Inc.",Medical Instruments & Supplies,0.061500,0.006000,0.069750,105.00,791.30,30650.00
5,AGTC,2,Applied Genetic Technologies Corporation,Biotechnology,5.250000,3.940000,5.420000,-1.87,27.27,17.45
6,OCGN,2,"Ocugen, Inc.",Biotechnology,15.810000,2.010000,18.770000,70.37,660.10,512.79
7,XOM,2,Exxon Mobil Corporation,Oil & Gas Integrated,51.230000,44.523884,51.485660,2.32,13.73,NaN
8,REI,2,"Ring Energy, Inc.",Oil & Gas E&P,2.300000,1.140000,2.390000,14.43,90.08,112.96
9,BIOL,2,"BIOLASE, Inc.",Medical Devices,1.360000,1.020000,1.450000,11.48,27.10,147.27


In [ ]:
_posts

,id,title,score,comments,upvote_ratio,total_awards,flair,author_reliability,Tickers
0,lg0b72,"$AKBA Bio-pharma stock DD, upcoming catalyst",1,1,1.00,0,DD :DD:,0.941098,{AKBA}
1,lg07u2,Improved: Algorithm that shows currently trend...,3,2,0.80,0,Discussion,3.340125,{}
2,lfzgro,GRPS is now Pink!!,6,6,0.80,1,Catalyst,1.190520,{GRPS}
3,lfytwy,"[DD] On Abeona Therapeutics $ABEO, with upcomi...",26,12,0.94,0,DD :DD:,1.354990,{ABEO}
4,lfyikp,WALRF good swing play. (EV play),5,4,0.73,0,General Discussion,4.731350,{WALRF}
...,...,...,...,...,...,...,...,...,...
121,lenmhf,(GTII) The Great OTC SHORT SQUEEZE of 2021,49,70,0.89,2,DD :DD:,0.908363,{GTII}
122,len9kc,HIPH - Catalyst w DD,11,10,0.76,0,Catalyst,11.450975,{HIPH}
123,lemwn2,ALYI and Hemp batteries.,15,17,0.83,0,General Discussion,2.661775,{ALYI}
124,lemkx4,Undiscovered Solar Play,9,15,0.63,0,Stock Info :stonk:,3.244850,{}


In [ ]:
_posts.Tickers = _posts.Tickers.apply(list)

In [ ]:
_posts.head()

,id,title,score,comments,upvote_ratio,total_awards,flair,author_reliability,Tickers
0,lg0b72,"$AKBA Bio-pharma stock DD, upcoming catalyst",1,1,1.00,0,DD :DD:,0.941098,[AKBA]
1,lg07u2,Improved: Algorithm that shows currently trend...,3,2,0.80,0,Discussion,3.340125,[]
2,lfzgro,GRPS is now Pink!!,6,6,0.80,1,Catalyst,1.190520,[GRPS]
3,lfytwy,"[DD] On Abeona Therapeutics $ABEO, with upcomi...",26,12,0.94,0,DD :DD:,1.354990,[ABEO]
4,lfyikp,WALRF good swing play. (EV play),5,4,0.73,0,General Discussion,4.731350,[WALRF]


In [ ]:
res = _posts.set_index(["id",	"title",	"score",	"comments",	"upvote_ratio",	"total_awards",	"flair",	"author_reliability"])['Tickers'].apply(pd.Series).stack()

In [ ]:
res = res.reset_index()

In [ ]:
res.rename(columns={0: 'Tickers'}, inplace=True)
res

,id,title,score,comments,upvote_ratio,total_awards,flair,author_reliability,level_8,Tickers
0,lg0b72,"$AKBA Bio-pharma stock DD, upcoming catalyst",1,1,1.00,0,DD :DD:,0.941098,0,AKBA
1,lfzgro,GRPS is now Pink!!,6,6,0.80,1,Catalyst,1.190520,0,GRPS
2,lfytwy,"[DD] On Abeona Therapeutics $ABEO, with upcomi...",26,12,0.94,0,DD :DD:,1.354990,0,ABEO
3,lfyikp,WALRF good swing play. (EV play),5,4,0.73,0,General Discussion,4.731350,0,WALRF
4,lfy24l,$NAKD DD BY HEDGE FUND ANALYST,4,4,0.70,0,DD/Research,4.958325,0,NAKD
...,...,...,...,...,...,...,...,...,...,...
97,leo92y,MINE -Minerco in the Psychedelic Sector,43,29,0.91,0,General Discussion,0.831550,0,MINE
98,lenmhf,(GTII) The Great OTC SHORT SQUEEZE of 2021,49,70,0.89,2,DD :DD:,0.908363,0,GTII
99,len9kc,HIPH - Catalyst w DD,11,10,0.76,0,Catalyst,11.450975,0,HIPH
100,lemwn2,ALYI and Hemp batteries.,15,17,0.83,0,General Discussion,2.661775,0,ALYI


In [ ]:
res["rating"] = (res["score"]+res["comments"]+res["total_awards"])*res["upvote_ratio"]
res = res.sort_values(by=["rating"], ascending=False)

In [ ]:
res.head()

,id,title,score,comments,upvote_ratio,total_awards,flair,author_reliability,level_8,Tickers,rating
55,lfeqjo,NASDAQ: $ZSAN - Zosano Pharma Corp. - The COVI...,5979,1203,0.92,117,DD :DD:,3.799975,0,ZSAN,6715.08
71,lfargh,European Metals ($ERPNF) - Speculating on Euro...,1729,431,0.97,45,Catalyst,7.094625,0,ERPNF,2138.85
27,lfmvdh,"Talon Metals $TLOFF is very likely to go 10x, ...",1184,460,0.96,31,DD :DD:,0.765520,0,TLOFF,1608.00
9,lfvrxo,ADVICE for NEW and SERIOUS TRADERS,895,140,0.98,34,General Discussion,0.862275,0,NEW,1047.62
43,lfjtbn,$TBLT (SWING) - DD post. Can be a multibagger ...,543,208,0.96,12,DD :DD:,3.300700,0,TBLT,732.48


In [ ]:
res = res[res.flair.str.contains("DD|Catalyst").astype('bool')]

In [ ]:
res

,id,title,score,comments,upvote_ratio,total_awards,flair,author_reliability,level_8,Tickers,rating
55,lfeqjo,NASDAQ: $ZSAN - Zosano Pharma Corp. - The COVI...,5979,1203,0.92,117,DD :DD:,3.799975,0,ZSAN,6715.08
71,lfargh,European Metals ($ERPNF) - Speculating on Euro...,1729,431,0.97,45,Catalyst,7.094625,0,ERPNF,2138.85
27,lfmvdh,"Talon Metals $TLOFF is very likely to go 10x, ...",1184,460,0.96,31,DD :DD:,0.765520,0,TLOFF,1608.00
43,lfjtbn,$TBLT (SWING) - DD post. Can be a multibagger ...,543,208,0.96,12,DD :DD:,3.300700,0,TBLT,732.48
53,lfewgi,NASDAQ: $ZSAN - Zosano Pharma Corp. - The Comp...,574,136,0.99,13,DD/Research,3.799975,0,ZSAN,715.77
...,...,...,...,...,...,...,...,...,...,...,...
5,lfxzvu,Novabay Pharma ($NBY) up 35% from last week an...,2,2,0.60,0,Catalyst,2.348350,0,NBY,2.40
0,lg0b72,"$AKBA Bio-pharma stock DD, upcoming catalyst",1,1,1.00,0,DD :DD:,0.941098,0,AKBA,2.00
59,lfdxp6,"Parcelpal Technologies, PTNYF (OTC), PKG.CN (CSE)",0,1,0.27,0,DD/Research,1.438800,2,PTNYF,0.27
58,lfdxp6,"Parcelpal Technologies, PTNYF (OTC), PKG.CN (CSE)",0,1,0.27,0,DD/Research,1.438800,1,CN,0.27


In [ ]:
res = res.drop(columns=["score", "comments", "upvote_ratio", "total_awards", "level_8"])

In [ ]:
catalyst_df = res[res.flair.str.contains("Catalyst").astype('bool')]
dd_df = res[res.flair.str.contains("DD").astype('bool')]

In [ ]:
catalyst_df.drop_duplicates(subset='Tickers', keep="first", inplace=True)
dd_df.drop_duplicates(subset='Tickers', keep="first", inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
final_df = tick_df.copy()

In [ ]:
final_df["DD"] =   "["+final_df.Ticker.map(dd_df.set_index('Tickers')['title'].to_dict())+"](https://reddit.com/"+final_df.Ticker.map(dd_df.set_index('Tickers')['id'].to_dict())+")"

In [ ]:
final_df["Catalyst"] = "["+final_df.Ticker.map(catalyst_df.set_index('Tickers')['title'].to_dict())+"](https://reddit.com/"+final_df.Ticker.map(catalyst_df.set_index('Tickers')['id'].to_dict())+")"

In [ ]:
final_df.fillna("N/A", inplace=True)

In [ ]:
final_df["On RobinHood"] = final_df.Ticker.apply(lambda t: "No" if yf.Ticker(t).info["exchange"] == "PNK" else "Yes")

In [ ]:
final_df["Rating"] = final_df.Ticker.map(dd_df.set_index('Tickers')['rating'].to_dict())
final_df["Author Reliability"] = final_df.Ticker.map(dd_df.set_index('Tickers')['author_reliability'].to_dict())


In [ ]:
final_df

,Ticker,Mentions,Name,Industry,Previous Close,5d Low,5d High,1d Change (%),5d Change (%),1mo Change (%),DD,Catalyst,On RobinHood,Rating,Author Reliability
0,OZSC,3,"Ozop Energy Solutions, Inc.",Electrical Equipment & Parts,0.378400,0.148000,0.385000,51.42,122.07,2456.76,N/A,[OZSC hires new Director of Operations](https:...,No,NaN,NaN
1,ZSAN,3,Zosano Pharma Corporation,Biotechnology,1.930000,0.940000,1.950000,72.32,93.97,129.76,[NASDAQ: $ZSAN - Zosano Pharma Corp. - The COV...,N/A,Yes,6715.08,3.799975
2,EAST,3,"Eastside Distilling, Inc.",Beverages—Wineries & Distilleries,1.770000,1.430000,1.940000,-0.62,18.79,27.34,[Eastside Distilling - $EAST](https://reddit.c...,N/A,Yes,63.36,1.194338
3,TNXP,2,Tonix Pharmaceuticals Holding Corp.,Biotechnology,1.550000,0.950000,1.660000,-6.63,56.57,68.48,N/A,[Tonix Pharmaceuticals (TNXP) to Develop COVID...,Yes,NaN,NaN
4,INND,2,"InnerScope Hearing Technologies, Inc.",Medical Instruments & Supplies,0.061500,0.006000,0.069750,105.00,791.30,30650,N/A,N/A,No,NaN,NaN
5,AGTC,2,Applied Genetic Technologies Corporation,Biotechnology,5.250000,3.940000,5.420000,-1.87,27.27,17.45,N/A,N/A,Yes,NaN,NaN
6,OCGN,2,"Ocugen, Inc.",Biotechnology,15.810000,2.010000,18.770000,70.37,660.10,512.79,N/A,N/A,Yes,NaN,NaN
7,XOM,2,Exxon Mobil Corporation,Oil & Gas Integrated,51.230000,44.523884,51.485660,2.32,13.73,N/A,[$XOM (Exxon Mobil) and $REI (Ring Energy) pos...,N/A,Yes,34.32,14.893960
8,REI,2,"Ring Energy, Inc.",Oil & Gas E&P,2.300000,1.140000,2.390000,14.43,90.08,112.96,[$XOM (Exxon Mobil) and $REI (Ring Energy) pos...,N/A,Yes,34.32,14.893960
9,BIOL,2,"BIOLASE, Inc.",Medical Devices,1.360000,1.020000,1.450000,11.48,27.10,147.27,[$BIOL DD - Stocks With Frickin' Laser Beams](...,N/A,Yes,606.72,1.695750


In [ ]:
final_df = final_df[final_df["Previous Close"] < 7]

In [ ]:
final_df = final_df[["Ticker", "Mentions", "Name", "Industry", "On RobinHood", "Previous Close", "DD", "Catalyst", "Author Reliability", "Rating", "5d Low", "5d High", "1d Change (%)", "5d Change (%)", "1mo Change (%)"]]

In [ ]:
final_df

,Ticker,Mentions,Name,Industry,On RobinHood,Previous Close,DD,Catalyst,Author Reliability,Rating,5d Low,5d High,1d Change (%),5d Change (%),1mo Change (%)
0,OZSC,3,"Ozop Energy Solutions, Inc.",Electrical Equipment & Parts,No,0.3784,N/A,[OZSC hires new Director of Operations](https:...,NaN,NaN,0.1480,0.38500,51.42,122.07,2456.76
1,ZSAN,3,Zosano Pharma Corporation,Biotechnology,Yes,1.9300,[NASDAQ: $ZSAN - Zosano Pharma Corp. - The COV...,N/A,3.799975,6715.08,0.9400,1.95000,72.32,93.97,129.76
2,EAST,3,"Eastside Distilling, Inc.",Beverages—Wineries & Distilleries,Yes,1.7700,[Eastside Distilling - $EAST](https://reddit.c...,N/A,1.194338,63.36,1.4300,1.94000,-0.62,18.79,27.34
3,TNXP,2,Tonix Pharmaceuticals Holding Corp.,Biotechnology,Yes,1.5500,N/A,[Tonix Pharmaceuticals (TNXP) to Develop COVID...,NaN,NaN,0.9500,1.66000,-6.63,56.57,68.48
4,INND,2,"InnerScope Hearing Technologies, Inc.",Medical Instruments & Supplies,No,0.0615,N/A,N/A,NaN,NaN,0.0060,0.06975,105.00,791.30,30650
5,AGTC,2,Applied Genetic Technologies Corporation,Biotechnology,Yes,5.2500,N/A,N/A,NaN,NaN,3.9400,5.42000,-1.87,27.27,17.45
8,REI,2,"Ring Energy, Inc.",Oil & Gas E&P,Yes,2.3000,[$XOM (Exxon Mobil) and $REI (Ring Energy) pos...,N/A,14.893960,34.32,1.1400,2.39000,14.43,90.08,112.96
9,BIOL,2,"BIOLASE, Inc.",Medical Devices,Yes,1.3600,[$BIOL DD - Stocks With Frickin' Laser Beams](...,N/A,1.695750,606.72,1.0200,1.45000,11.48,27.10,147.27
10,PTNYF,2,ParcelPal Technology Inc.,Software—Application,No,0.2124,"[Parcelpal Technologies, PTNYF (OTC), PKG.CN (...",N/A,1.438800,23.24,0.1050,0.22900,21.37,85.96,50.35
12,NAKD,2,Naked Brand Group Limited,Apparel Manufacturing,Yes,1.1400,[$NAKD DD BY HEDGE FUND ANALYST](https://reddi...,N/A,4.958325,7.80,0.7400,1.47000,-6.17,-3.39,232.36


In [ ]:
final_df.fillna("N/A", inplace=True)

In [ ]:
final_df["Author Reliability"] = final_df["Author Reliability"].apply(lambda x: round(x, 2) if x != "N/A" else "N/A")


In [ ]:
final_df.to_markdown(index=False)

"| Ticker   |   Mentions | Name                                              | Industry                           | On RobinHood   |   Previous Close | DD                                                                                                                        | Catalyst                                                                                                                                                           | Author Reliability   | Rating             |   5d Low |   5d High |   1d Change (%) |   5d Change (%) |   1mo Change (%) |\n|:---------|-----------:|:--------------------------------------------------|:-----------------------------------|:---------------|-----------------:|:--------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------